<a href="https://colab.research.google.com/github/XinyaoWa/hydroai-colab/blob/main/Use_RecDP_to_run_TwitterRecSys2021_data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyspark==3.0.0

In [3]:
pip install pyrecdp

In [30]:
pip install transformers

     |████████████████████████████████| 3.5 MB 27.0 MB/s 
     |████████████████████████████████| 6.8 MB 49.3 MB/s 
     |████████████████████████████████| 895 kB 46.3 MB/s 
     |████████████████████████████████| 596 kB 47.0 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [16]:
import pyrecdp
import gc,time,os
import pandas as pd
import numpy as np
from pyspark.sql import *
from pyspark import *
import pyspark.sql.functions as f
import pyspark.sql.types as t
from timeit import default_timer as timer
import logging
from pyrecdp.data_processor import *
from pyrecdp.encoder import *
from pyrecdp.utils import *
import hashlib
import re

In [7]:
!mkdir recsys2021
os.chdir('recsys2021')
!mkdir spark_tmp
!mkdir data
!ls

mkdir: cannot create directory ‘recsys2021’: File exists


In [9]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jm_bqkzV1CYgH885iaYsDoal1N8J-sE6' -O original_TwitterRecSys2021_sample.parquet

--2022-01-28 13:50:39--  https://docs.google.com/uc?export=download&id=1jm_bqkzV1CYgH885iaYsDoal1N8J-sE6
Resolving docs.google.com (docs.google.com)... 142.250.73.206, 2607:f8b0:4004:829::200e
Connecting to docs.google.com (docs.google.com)|142.250.73.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/89mb4ivdkgrer8aoiqcmur511f0shc7r/1643377800000/18016102022855825607/*/1jm_bqkzV1CYgH885iaYsDoal1N8J-sE6?e=download [following]
--2022-01-28 13:50:40--  https://doc-00-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/89mb4ivdkgrer8aoiqcmur511f0shc7r/1643377800000/18016102022855825607/*/1jm_bqkzV1CYgH885iaYsDoal1N8J-sE6?e=download
Resolving doc-00-1s-docs.googleusercontent.com (doc-00-1s-docs.googleusercontent.com)... 142.250.81.193, 2607:f8b0:4004:82f::2001
Connecting to doc-00-1s-docs.googleusercontent.com (doc-00-1s-docs.googleusercon

In [23]:
path_prefix = './'
current_path = 'data/'
dicts_folder = "recsys_dicts/"

In [14]:
##### 1. Start spark and initialize data processor #####
recdp_path = pyrecdp.__path__[0]
# scala_udf_jars = recdp_path + "/ScalaProcessUtils/built/30/recdp-scala-extensions-0.1.0-jar-with-dependencies.jar"
scala_udf_jars = recdp_path + "/ScalaProcessUtils/target/recdp-scala-extensions-0.1.0-jar-with-dependencies.jar"

In [19]:
spark = SparkSession.builder.master('local')\
    .appName("Recsys2021_data_process")\
    .config("spark.driver.memory", '50g')\
    .config("spark.local.dir", path_prefix+"/spark_tmp")\
    .config("spark.sql.broadcastTimeout", "7200")\
    .config("spark.cleaner.periodicGC.interval", "15min")\
    .config("spark.executorEnv.HF_DATASETS_OFFLINE", "1")\
    .config("spark.executorEnv.TRANSFORMERS_OFFLINE", "1")\
    .config("spark.driver.extraClassPath", f"{scala_udf_jars}")\
    .config("spark.executor.extraClassPath", f"{scala_udf_jars}")\
    .config("spark.sql.adaptive.enabled", True)\
    .config("spark.driver.maxResultSize","16g")\
    .config("spark.sql.codegen.maxFields", 300)\
    .getOrCreate()

In [24]:
proc = DataProcessor(spark, path_prefix,
                    current_path=current_path, dicts_path=dicts_folder, shuffle_disk_capacity="300GB",spark_mode='local')

recdp-scala-extension is enabled
per core memory size is 25.000 GB and shuffle_disk maximum capacity is 300.000 GB


In [25]:
df = spark.read.parquet(path_prefix + "original_TwitterRecSys2021_sample.parquet")

In [26]:
df = df.withColumnRenamed('enaging_user_following_count', 'engaging_user_following_count')
df = df.withColumnRenamed('enaging_user_is_verified', 'engaging_user_is_verified')
df = df.drop("tokens")
gc.collect()
print("data loaded!")

data loaded!


In [39]:
target_list = [
    'reply_timestamp',
    'retweet_timestamp',
    'retweet_with_comment_timestamp',
    'like_timestamp'
    ]
indexlist = ["engaging_user_id","tweet_id"]
stage1_features_list = [
    'engaged_with_user_follower_count',
    'engaged_with_user_following_count',
    'engaged_with_user_is_verified',
    'engaging_user_follower_count',
    'engaging_user_following_count',
    'engaging_user_is_verified',
    'has_photo',
    'has_video',
    'has_gif',
    'a_ff_rate',
    'b_ff_rate', 
    'dt_hour',
    'dt_dow',
    'has_mention',  
    'mentioned_bucket_id',    
    'mentioned_count',    
    'most_used_word_bucket_id',
    'second_used_word_bucket_id',
    'TE_tweet_type_reply_timestamp',
    'TE_tweet_type_retweet_timestamp',
    'TE_dt_dow_retweet_timestamp',
    'TE_most_used_word_bucket_id_reply_timestamp',
    'TE_most_used_word_bucket_id_retweet_timestamp',
    'TE_most_used_word_bucket_id_retweet_with_comment_timestamp',
    'TE_most_used_word_bucket_id_like_timestamp',
    'TE_second_used_word_bucket_id_reply_timestamp',
    'TE_second_used_word_bucket_id_retweet_timestamp',
    'TE_second_used_word_bucket_id_retweet_with_comment_timestamp',
    'TE_second_used_word_bucket_id_like_timestamp',
    'TE_mentioned_bucket_id_retweet_timestamp',
    'TE_mentioned_bucket_id_retweet_with_comment_timestamp',
    'TE_mentioned_bucket_id_like_timestamp',
    'TE_mentioned_bucket_id_reply_timestamp',
    'TE_language_reply_timestamp',
    'TE_language_retweet_timestamp',
    'TE_language_retweet_with_comment_timestamp',
    'TE_language_like_timestamp',
    'TE_mentioned_count_reply_timestamp',
    'TE_mentioned_count_retweet_timestamp',
    'TE_mentioned_count_retweet_with_comment_timestamp',
    'TE_mentioned_count_like_timestamp',
    'TE_engaged_with_user_id_reply_timestamp',
    'TE_engaged_with_user_id_retweet_timestamp',
    'TE_engaged_with_user_id_retweet_with_comment_timestamp',
    'TE_engaged_with_user_id_like_timestamp',
    'GTE_language_engaged_with_user_id_reply_timestamp',
    'GTE_language_engaged_with_user_id_retweet_timestamp',
    'GTE_language_engaged_with_user_id_retweet_with_comment_timestamp',
    'GTE_language_engaged_with_user_id_like_timestamp',
    'GTE_tweet_type_engaged_with_user_id_reply_timestamp',
    'GTE_tweet_type_engaged_with_user_id_retweet_timestamp',
    'GTE_tweet_type_engaged_with_user_id_retweet_with_comment_timestamp',
    'GTE_tweet_type_engaged_with_user_id_like_timestamp',
    'GTE_has_mention_engaging_user_id_reply_timestamp',
    'GTE_has_mention_engaging_user_id_retweet_timestamp',
    'GTE_has_mention_engaging_user_id_retweet_with_comment_timestamp',
    'GTE_has_mention_engaging_user_id_like_timestamp',
    'GTE_mentioned_bucket_id_engaging_user_id_reply_timestamp',
    'GTE_mentioned_bucket_id_engaging_user_id_retweet_timestamp',
    'GTE_mentioned_bucket_id_engaging_user_id_retweet_with_comment_timestamp',
    'GTE_mentioned_bucket_id_engaging_user_id_like_timestamp',
    'GTE_language_engaging_user_id_reply_timestamp',
    'GTE_language_engaging_user_id_retweet_timestamp',
    'GTE_language_engaging_user_id_retweet_with_comment_timestamp',
    'GTE_language_engaging_user_id_like_timestamp',
    'GTE_tweet_type_engaging_user_id_reply_timestamp',
    'GTE_tweet_type_engaging_user_id_retweet_timestamp',
    'GTE_tweet_type_engaging_user_id_retweet_with_comment_timestamp',
    'GTE_tweet_type_engaging_user_id_like_timestamp',
    'GTE_dt_dow_engaged_with_user_id_reply_timestamp',
    'GTE_dt_dow_engaged_with_user_id_retweet_timestamp',
    'GTE_dt_dow_engaged_with_user_id_retweet_with_comment_timestamp',
    'GTE_dt_dow_engaged_with_user_id_like_timestamp',
    'GTE_mentioned_count_engaging_user_id_reply_timestamp',
    'GTE_mentioned_count_engaging_user_id_retweet_timestamp',
    'GTE_mentioned_count_engaging_user_id_retweet_with_comment_timestamp',
    'GTE_mentioned_count_engaging_user_id_like_timestamp',
    'GTE_dt_hour_engaged_with_user_id_reply_timestamp',
    'GTE_dt_hour_engaged_with_user_id_retweet_timestamp',
    'GTE_dt_hour_engaged_with_user_id_retweet_with_comment_timestamp',
    'GTE_dt_hour_engaged_with_user_id_like_timestamp',
    'GTE_dt_dow_engaging_user_id_reply_timestamp',
    'GTE_dt_dow_engaging_user_id_retweet_timestamp',
    'GTE_dt_dow_engaging_user_id_retweet_with_comment_timestamp',
    'GTE_dt_dow_engaging_user_id_like_timestamp',
    'GTE_dt_hour_engaging_user_id_reply_timestamp',
    'GTE_dt_hour_engaging_user_id_retweet_timestamp',
    'GTE_dt_hour_engaging_user_id_retweet_with_comment_timestamp',
    'GTE_dt_hour_engaging_user_id_like_timestamp',
    "engagee_follows_engager",
    "dt_minute",
    "len_domains",
    "len_hashtags",
    "len_links",
    "TE_tweet_type_retweet_with_comment_timestamp",
    "TE_tweet_type_like_timestamp",
    "GTE_engaged_with_user_id_engaging_user_id_reply_timestamp",
    "GTE_engaged_with_user_id_engaging_user_id_retweet_timestamp",
    "GTE_engaged_with_user_id_engaging_user_id_retweet_with_comment_timestamp",
    "GTE_engaged_with_user_id_engaging_user_id_like_timestamp",
    "GTE_engaged_with_user_id_language_tweet_type_reply_timestamp",
    "GTE_engaged_with_user_id_language_tweet_type_retweet_timestamp",
    "GTE_engaged_with_user_id_language_tweet_type_retweet_with_comment_timestamp",
    "GTE_engaged_with_user_id_language_tweet_type_like_timestamp",
    "GTE_engaging_user_id_language_tweet_type_reply_timestamp",
    "GTE_engaging_user_id_language_tweet_type_retweet_timestamp",
    "GTE_engaging_user_id_language_tweet_type_retweet_with_comment_timestamp",
    "GTE_engaging_user_id_language_tweet_type_like_timestamp",
    "TE_engaging_user_id_reply_timestamp",
    "TE_engaging_user_id_retweet_timestamp",
    "TE_engaging_user_id_retweet_with_comment_timestamp",
    "TE_engaging_user_id_like_timestamp",
    "GTE_language_tweet_type_present_media_reply_timestamp",
    "GTE_language_tweet_type_present_media_retweet_timestamp",
    "GTE_language_tweet_type_present_media_retweet_with_comment_timestamp",
    "GTE_language_tweet_type_present_media_like_timestamp",
    "TE_present_media_reply_timestamp",
    "TE_present_media_retweet_timestamp",
    "TE_present_media_retweet_with_comment_timestamp",
    "TE_present_media_like_timestamp",
    'TE_tw_word0_reply_timestamp', 
    'TE_tw_word0_retweet_timestamp', 
    'TE_tw_word0_retweet_with_comment_timestamp', 
    'TE_tw_word0_like_timestamp',
    "len_media",
    "ab_age_dff",
    "ab_age_rate",
    "ab_fing_rate",
    "ab_fer_rate",
    'GTE_engaging_user_is_verified_tweet_type_reply_timestamp', 
    'GTE_engaging_user_is_verified_tweet_type_retweet_timestamp', 
    'GTE_engaging_user_is_verified_tweet_type_retweet_with_comment_timestamp', 
    'GTE_engaging_user_is_verified_tweet_type_like_timestamp',
    'GTE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_reply_timestamp', 
    'GTE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_timestamp', 
    'GTE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_retweet_with_comment_timestamp', 
    'GTE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified_like_timestamp',
    'GTE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_reply_timestamp', 
    'GTE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_timestamp', 
    'GTE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_retweet_with_comment_timestamp', 
    'GTE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager_like_timestamp',
    'GTE_tw_original_user0_tweet_type_language_reply_timestamp', 
    'GTE_tw_original_user0_tweet_type_language_retweet_timestamp', 
    'GTE_tw_original_user0_tweet_type_language_retweet_with_comment_timestamp', 
    'GTE_tw_original_user0_tweet_type_language_like_timestamp',
    'GTE_tw_original_user1_tweet_type_language_reply_timestamp', 
    'GTE_tw_original_user1_tweet_type_language_retweet_timestamp', 
    'GTE_tw_original_user1_tweet_type_language_retweet_with_comment_timestamp', 
    'GTE_tw_original_user1_tweet_type_language_like_timestamp'
    ]
stage2_features_list = [
    'stage2_TE_engaged_with_user_id_reply_timestamp',
    'stage2_TE_engaged_with_user_id_retweet_timestamp',
    'stage2_TE_engaged_with_user_id_retweet_with_comment_timestamp',
    'stage2_TE_engaged_with_user_id_like_timestamp',
    'stage2_TE_language_reply_timestamp',
    'stage2_TE_language_retweet_timestamp',
    'stage2_TE_language_retweet_with_comment_timestamp',
    'stage2_TE_language_like_timestamp',
    'stage2_TE_dt_dow_retweet_timestamp',
    'stage2_TE_tweet_type_reply_timestamp',
    'stage2_TE_tweet_type_retweet_timestamp',
    'stage2_TE_most_used_word_bucket_id_reply_timestamp',
    'stage2_TE_most_used_word_bucket_id_retweet_timestamp',
    'stage2_TE_most_used_word_bucket_id_retweet_with_comment_timestamp',
    'stage2_TE_most_used_word_bucket_id_like_timestamp',
    'stage2_TE_second_used_word_bucket_id_reply_timestamp',
    'stage2_TE_second_used_word_bucket_id_retweet_timestamp',
    'stage2_TE_second_used_word_bucket_id_retweet_with_comment_timestamp',
    'stage2_TE_second_used_word_bucket_id_like_timestamp',
    'stage2_TE_mentioned_count_reply_timestamp',
    'stage2_TE_mentioned_count_retweet_timestamp',
    'stage2_TE_mentioned_count_retweet_with_comment_timestamp',
    'stage2_TE_mentioned_count_like_timestamp',
    'stage2_TE_mentioned_bucket_id_reply_timestamp',
    'stage2_TE_mentioned_bucket_id_retweet_timestamp',
    'stage2_TE_mentioned_bucket_id_retweet_with_comment_timestamp',
    'stage2_TE_mentioned_bucket_id_like_timestamp',
    'stage2_GTE_has_mention_engaging_user_id_reply_timestamp',
    'stage2_GTE_has_mention_engaging_user_id_retweet_timestamp',
    'stage2_GTE_has_mention_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_has_mention_engaging_user_id_like_timestamp',
    'stage2_GTE_mentioned_count_engaging_user_id_reply_timestamp',
    'stage2_GTE_mentioned_count_engaging_user_id_retweet_timestamp',
    'stage2_GTE_mentioned_count_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_mentioned_count_engaging_user_id_like_timestamp',
    'stage2_GTE_mentioned_bucket_id_engaging_user_id_reply_timestamp',
    'stage2_GTE_mentioned_bucket_id_engaging_user_id_retweet_timestamp',
    'stage2_GTE_mentioned_bucket_id_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_mentioned_bucket_id_engaging_user_id_like_timestamp',
    'stage2_GTE_language_engaged_with_user_id_reply_timestamp',
    'stage2_GTE_language_engaged_with_user_id_retweet_timestamp',
    'stage2_GTE_language_engaged_with_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_language_engaged_with_user_id_like_timestamp',
    'stage2_GTE_language_engaging_user_id_reply_timestamp',
    'stage2_GTE_language_engaging_user_id_retweet_timestamp',
    'stage2_GTE_language_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_language_engaging_user_id_like_timestamp',
    'stage2_GTE_dt_dow_engaged_with_user_id_reply_timestamp',
    'stage2_GTE_dt_dow_engaged_with_user_id_retweet_timestamp',
    'stage2_GTE_dt_dow_engaged_with_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_dt_dow_engaged_with_user_id_like_timestamp',
    'stage2_GTE_dt_dow_engaging_user_id_reply_timestamp',
    'stage2_GTE_dt_dow_engaging_user_id_retweet_timestamp',
    'stage2_GTE_dt_dow_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_dt_dow_engaging_user_id_like_timestamp',
    'stage2_GTE_dt_hour_engaged_with_user_id_reply_timestamp',
    'stage2_GTE_dt_hour_engaged_with_user_id_retweet_timestamp',
    'stage2_GTE_dt_hour_engaged_with_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_dt_hour_engaged_with_user_id_like_timestamp',
    'stage2_GTE_dt_hour_engaging_user_id_reply_timestamp',
    'stage2_GTE_dt_hour_engaging_user_id_retweet_timestamp',
    'stage2_GTE_dt_hour_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_dt_hour_engaging_user_id_like_timestamp',
    'stage2_GTE_tweet_type_engaged_with_user_id_reply_timestamp',
    'stage2_GTE_tweet_type_engaged_with_user_id_retweet_timestamp',
    'stage2_GTE_tweet_type_engaged_with_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_tweet_type_engaged_with_user_id_like_timestamp',
    'stage2_GTE_tweet_type_engaging_user_id_reply_timestamp',
    'stage2_GTE_tweet_type_engaging_user_id_retweet_timestamp',
    'stage2_GTE_tweet_type_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_tweet_type_engaging_user_id_like_timestamp',
    'stage2_GTE_engaged_with_user_id_engaging_user_id_reply_timestamp',
    'stage2_GTE_engaged_with_user_id_engaging_user_id_retweet_timestamp',
    'stage2_GTE_engaged_with_user_id_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_GTE_engaged_with_user_id_engaging_user_id_like_timestamp',
    'stage2_GTE_engaged_with_user_id_language_tweet_type_reply_timestamp',
    'stage2_GTE_engaged_with_user_id_language_tweet_type_retweet_timestamp',
    'stage2_GTE_engaged_with_user_id_language_tweet_type_retweet_with_comment_timestamp',
    'stage2_GTE_engaged_with_user_id_language_tweet_type_like_timestamp',
    'stage2_GTE_engaging_user_id_language_tweet_type_reply_timestamp',
    'stage2_GTE_engaging_user_id_language_tweet_type_retweet_timestamp',
    'stage2_GTE_engaging_user_id_language_tweet_type_retweet_with_comment_timestamp',
    'stage2_GTE_engaging_user_id_language_tweet_type_like_timestamp',
    'stage2_TE_engaging_user_id_reply_timestamp',
    'stage2_TE_engaging_user_id_retweet_timestamp',
    'stage2_TE_engaging_user_id_retweet_with_comment_timestamp',
    'stage2_TE_engaging_user_id_like_timestamp',
    'stage2_GTE_language_tweet_type_present_media_reply_timestamp',
    'stage2_GTE_language_tweet_type_present_media_retweet_timestamp',
    'stage2_GTE_language_tweet_type_present_media_retweet_with_comment_timestamp',
    'stage2_GTE_language_tweet_type_present_media_like_timestamp',
    'stage2_TE_present_media_reply_timestamp',
    'stage2_TE_present_media_retweet_timestamp',
    'stage2_TE_present_media_retweet_with_comment_timestamp',
    'stage2_TE_present_media_like_timestamp',
    'stage2_TE_tw_word0_reply_timestamp',
    'stage2_TE_tw_word0_retweet_timestamp',
    'stage2_TE_tw_word0_retweet_with_comment_timestamp',
    'stage2_TE_tw_word0_like_timestamp',
    'stage2_TE_tweet_id_reply_timestamp',
    'stage2_TE_tweet_id_retweet_timestamp',
    'stage2_TE_tweet_id_retweet_with_comment_timestamp',
    'stage2_TE_tweet_id_like_timestamp',
    'stage2_CE_engaged_with_user_id',
    'stage2_CE_engaging_user_id',
    'stage2_CE_language',
    'stage2_CE_present_media',
    'stage2_CE_tw_word0',
    'stage2_CE_tweet_id',
    'stage2_GCE_engaged_with_user_id_language_tweet_type',
    'stage2_GCE_engaging_user_id_language_tweet_type',
    'stage2_GCE_language_tweet_type_present_media',
    'stage2_GCE_engaged_with_user_id_engaging_user_id']

final_feature_list_stage1 = target_list + indexlist + stage1_features_list
final_feature_list_stage2 = final_feature_list_stage1 + stage2_features_list

TE_col_features_stage1 = [
            'engaged_with_user_id',
            'language',
            'dt_dow',
            'tweet_type',
            'most_used_word_bucket_id',
            'second_used_word_bucket_id',
            'mentioned_count',
            'mentioned_bucket_id',
            ['has_mention', 'engaging_user_id'],
            ['mentioned_count', 'engaging_user_id'],
            ['mentioned_bucket_id', 'engaging_user_id'],
            ['language', 'engaged_with_user_id'],
            ['language', 'engaging_user_id'],
            ['dt_dow', 'engaged_with_user_id'],
            ['dt_dow', 'engaging_user_id'],
            ['dt_hour', 'engaged_with_user_id'],
            ['dt_hour', 'engaging_user_id'],
            ['tweet_type', 'engaged_with_user_id'],
            ['tweet_type', 'engaging_user_id'],
            ['engaged_with_user_id','engaging_user_id'],
            ['engaged_with_user_id','language','tweet_type'],
            ['engaging_user_id','language','tweet_type'],
            'engaging_user_id',
            ['language','tweet_type','present_media'],
            'present_media',
            'tw_word0',
            ['engaging_user_is_verified','tweet_type'],
            ['present_domains', 'language', 'engagee_follows_engager', 'tweet_type', 'present_media', 'engaged_with_user_is_verified'],
            ['present_media', 'tweet_type', 'language', 'engaged_with_user_is_verified', 'engaging_user_is_verified', 'engagee_follows_engager'],
            ['tw_original_user0', 'tweet_type', 'language'],
            ['tw_original_user1', 'tweet_type', 'language'],
    ]
TE_col_features_stage1_threshold = {
    "TE_engaged_with_user_id": 0,
    "TE_language": 0,
    "TE_dt_dow": 0,
    "TE_tweet_type": 0,
    "TE_most_used_word_bucket_id": 0,
    "TE_second_used_word_bucket_id": 0,
    "TE_mentioned_count": 0,
    "TE_mentioned_bucket_id": 0,
    "GTE_has_mention_engaging_user_id": 2,
    "GTE_mentioned_count_engaging_user_id": 2,
    "GTE_mentioned_bucket_id_engaging_user_id": 2,
    "GTE_language_engaged_with_user_id": 1,
    "GTE_language_engaging_user_id": 2,
    "GTE_dt_dow_engaged_with_user_id": 2,
    "GTE_dt_dow_engaging_user_id": 4,
    "GTE_dt_hour_engaged_with_user_id": 3,
    "GTE_dt_hour_engaging_user_id": 4,
    "GTE_tweet_type_engaged_with_user_id": 1,
    "GTE_tweet_type_engaging_user_id": 3,
    "GTE_engaged_with_user_id_engaging_user_id": 3,
    "GTE_engaged_with_user_id_language_tweet_type": 1,
    "GTE_engaging_user_id_language_tweet_type": 3,
    "TE_engaging_user_id": 1,
    "GTE_language_tweet_type_present_media": 0,
    "TE_present_media": 0,
    "TE_tw_word0": 0,
    "GTE_engaging_user_is_verified_tweet_type": 0,
    "GTE_present_domains_language_engagee_follows_engager_tweet_type_present_media_engaged_with_user_is_verified": 0,
    "GTE_present_media_tweet_type_language_engaged_with_user_is_verified_engaging_user_is_verified_engagee_follows_engager": 0,
    "GTE_tw_original_user0_tweet_type_language": 0,
    "GTE_tw_original_user1_tweet_type_language": 0}
TE_col_features_stage2 = [
            'engaged_with_user_id',
            'language',
            'dt_dow',
            'tweet_type',
            'most_used_word_bucket_id',
            'second_used_word_bucket_id',
            'mentioned_count',
            'mentioned_bucket_id',
            ['has_mention', 'engaging_user_id'],
            ['mentioned_count', 'engaging_user_id'],
            ['mentioned_bucket_id', 'engaging_user_id'],
            ['language', 'engaged_with_user_id'],
            ['language', 'engaging_user_id'],
            ['dt_dow', 'engaged_with_user_id'],
            ['dt_dow', 'engaging_user_id'],
            ['dt_hour', 'engaged_with_user_id'],
            ['dt_hour', 'engaging_user_id'],
            ['tweet_type', 'engaged_with_user_id'],
            ['tweet_type', 'engaging_user_id'],
            ['engaged_with_user_id','engaging_user_id'],
            ['engaged_with_user_id','language','tweet_type'],
            ['engaging_user_id','language','tweet_type'],
            'engaging_user_id',
            ['language','tweet_type','present_media'],
            'present_media',
            'tw_word0',
            'tweet_id'
    ]
TE_col_excludes = {'dt_dow': ['reply_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']}
CE_col_features = ['engaged_with_user_id', 
                'engaging_user_id',
                'language',
                'present_media',
                'tw_word0',
                'tweet_id',
                ['engaged_with_user_id','language','tweet_type'],
                ['engaging_user_id','language','tweet_type'],
                ['language','tweet_type','present_media'],
                ['engaged_with_user_id','engaging_user_id']
    ]


In [28]:
def hashit(x):
    uhash = '0' if len(x)<=2 else x
    hash_object = hashlib.md5(uhash.encode('utf-8'))
    return int(hash_object.hexdigest(),16)%2**32

def ret_word( x, rw=0 ):
    x = x.split(' ')
    
    if len(x)>rw:
        return hashit(x[rw])
    elif rw<0:
        if len(x)>0:
            return hashit(x[-1])
        else:
            return 0
    else:
        return 0
    
def extract_hash(text, split_text='@', no=0):
    text = text.lower()
    uhash = ''
    text_split = text.split('@')
    if len(text_split)>(no+1):
        text_split = text_split[no+1].split(' ')
        cl_loop = True
        uhash += clean_text(text_split[0])
        while cl_loop:
            if len(text_split)>1:
                if text_split[1] in ['_']:
                    uhash += clean_text(text_split[1]) + clean_text(text_split[2])
                    text_split = text_split[2:]
                else:
                    cl_loop = False
            else:
                cl_loop = False
                
    return hashit(uhash)

def clean_text(text):
    if len(text)>1:
        if text[-1] in ['!', '?', ':', ';', '.', ',']:
            return(text[:-1])
    return(text)

def decodeBertTokenizerAndExtractFeatures(df, proc, output_name):
    from transformers import BertTokenizer
    tokenizer = BertTokenizer.from_pretrained(
        'bert-base-multilingual-cased', do_lower_case=False)

    op_fillna = FillNA(["present_domains","hashtags","present_links","present_media","tweet_type"],'')
    
    # define UDF
    tokenizer_decode = f.udf(lambda x: tokenizer.decode(
        [int(n) for n in x.split('\t')]))
    format_url = f.udf(lambda x: x.replace(
        'https : / / t. co / ', 'https://t.co/').replace('@ ', '@'))
    count_media = f.udf(lambda x: x.count('\t')+1 if x != '' else 0, t.IntegerType())
    udf_tw_word0 = f.udf(lambda x: ret_word(x,0), t.IntegerType())
    udf_tw_original_user0 = f.udf(lambda x: extract_hash(x, no=0))
    udf_tw_original_user1 = f.udf(lambda x: extract_hash(x, no=1))
    
    # define decode udf operations
    op_feature_modification_tokenizer_decode = FeatureAdd(
        cols={'tweet': 'text_tokens'}, udfImpl=tokenizer_decode)
    op_feature_modification_format_url = FeatureModification(
        cols=['tweet'], udfImpl=format_url)
    op_count_media = FeatureAdd(
        cols={'len_media': 'present_media'}, udfImpl=count_media)
    op_tw_word0 = FeatureAdd(
        cols={"tw_word0":"tweet"}, udfImpl=udf_tw_word0)
    op_tw_original_user0 = FeatureAdd(
        cols={'tw_original_user0': 'tweet'}, udfImpl=udf_tw_original_user0)
    op_tw_original_user1 = FeatureAdd(
        cols={'tw_original_user1': 'tweet'}, udfImpl=udf_tw_original_user1)
    
    op_feature_target_classify = FeatureModification(cols={
        "reply_timestamp": "f.when(f.col('reply_timestamp') > 0, 1).otherwise(0)",
        "retweet_timestamp": "f.when(f.col('retweet_timestamp') > 0, 1).otherwise(0)",
        "retweet_with_comment_timestamp": "f.when(f.col('retweet_with_comment_timestamp') > 0, 1).otherwise(0)",
        "like_timestamp": "f.when(f.col('like_timestamp') > 0, 1).otherwise(0)"}, op='inline')
    op_feature_dtype = FeatureModification(cols={
        "engagee_follows_engager": "f.col('engagee_follows_engager').cast(t.IntegerType())"
       }, op='inline')
    
    # define new features
    op_feature_from_original = FeatureAdd(
        cols={"has_photo": "f.col('present_media').contains('Photo').cast(t.IntegerType())",
              "has_video": "f.col('present_media').contains('Vedio').cast(t.IntegerType())",
              "has_gif": "f.col('present_media').contains('GIF').cast(t.IntegerType())",             
              "a_ff_rate": "f.col('engaged_with_user_following_count')/f.col('engaged_with_user_follower_count')",
              "b_ff_rate": "f.col('engaging_user_following_count') /f.col('engaging_user_follower_count')",
              "dt_dow": "f.dayofweek(f.from_unixtime(f.col('tweet_timestamp'))).cast(t.IntegerType())",
              "dt_hour": "f.hour(f.from_unixtime(f.col('tweet_timestamp'))).cast(t.IntegerType())",     
              "dt_minute": "f.minute(f.from_unixtime(f.col('tweet_timestamp'))).cast(t.IntegerType())",     
              "mention": "f.regexp_extract(f.col('tweet'), r'[^RT]\s@(\S+)', 1)",
              "has_mention": "(f.col('mention')!= '').cast(t.IntegerType())",
              "len_domains":"f.when(f.col('present_domains') == '', f.lit(0)).otherwise(f.size(f.split(f.col('present_domains'), '\t'))).cast(t.IntegerType())",
              "len_hashtags": "f.when(f.col('hashtags') == '', f.lit(0)).otherwise(f.size(f.split(f.col('hashtags'), '\t'))).cast(t.IntegerType())",
              "len_links": "f.when(f.col('present_links') == '', f.lit(0)).otherwise(f.size(f.split(f.col('present_links'), '\t'))).cast(t.IntegerType())",
              "ab_age_dff":"f.col('engaged_with_user_account_creation') - f.col('engaging_user_account_creation')",
              "ab_age_rate":"(f.col('engaged_with_user_account_creation')+129)/(f.col('engaging_user_account_creation')+129)",
              "ab_fing_rate":"f.col('engaged_with_user_following_count')/(1+f.col('engaging_user_following_count'))",
              "ab_fer_rate":"f.col('engaged_with_user_follower_count')/(1+f.col('engaging_user_follower_count'))",
        }, op='inline')
    
    # execute
    proc.reset_ops([op_fillna,
                    op_feature_modification_tokenizer_decode,
                    op_feature_modification_format_url,
                    op_count_media,
                    op_tw_word0,
                    op_tw_original_user0,
                    op_tw_original_user1,
                    op_feature_target_classify,
                    op_feature_dtype,
                    op_feature_from_original])
    t1 = timer()
    df = proc.transform(df, name=output_name)
    t2 = timer()
    print("BertTokenizer decode and feature extacting took %.3f" % (t2 - t1))

    return df


In [35]:
def categorifyFeatures(df, proc, output_name, gen_dict, sampleRatio=1):
    # 1. prepare dictionary
    dict_dfs = []
    if gen_dict:
        # only call below function when target dicts were not pre-prepared
        op_gen_dict_multiItems = GenerateDictionary(['tweet'], doSplit=True, sep=' ', bucketSize=100)
        op_gen_dict_singleItems = GenerateDictionary(['mention'], bucketSize=100)
        proc.reset_ops([op_gen_dict_multiItems, op_gen_dict_singleItems])
        t1 = timer()
        dict_dfs = proc.generate_dicts(df)
        t2 = timer()
        print("Generate Dictionary took %.3f" % (t2 - t1))
    else:
        dict_names = ['tweet', 'mention']
        dict_dfs = [{'col_name': name, 'dict': proc.spark.read.parquet(
            "%s/%s/%s/%s" % (proc.path_prefix, proc.current_path, proc.dicts_path, name))} for name in dict_names]
    # 2. since we need both mentioned_bucket_id and mentioned_count, add two mention id dict_dfs
    for dict_df in dict_dfs:
        if dict_df['col_name'] == 'mention':
            dict_dfs.append({'col_name': 'mentioned_bucket_id', 'dict':dict_df['dict']})
            dict_dfs.append({'col_name': 'mentioned_count', 'dict':dict_df['dict'].drop('dict_col_id').withColumnRenamed('count', 'dict_col_id')})
    op_feature_add = FeatureAdd({"mentioned_bucket_id": "f.col('mention')", "mentioned_count": "f.col('mention')"}, op='inline')
    
    # 3. categorify
    op_categorify_multiItems = Categorify([{'bucketized_tweet_word': 'tweet'}], dict_dfs=dict_dfs, doSplit=True, sep=' ')
    op_categorify_singleItem = Categorify(['mentioned_bucket_id', 'mentioned_count'], dict_dfs=dict_dfs)
    proc.reset_ops([op_feature_add, op_categorify_multiItems, op_categorify_singleItem])
    
    # 4. get most and second used bucketized_tweet_word
    op_feature_add_sorted_bucketized_tweet_word = FeatureAdd(
        cols={'sorted_bucketized_tweet_word': "f.expr('sortIntArrayByFrequency(bucketized_tweet_word)')"}, op='inline')
    op_feature_add_convert = FeatureAdd(
        cols={'most_used_word_bucket_id': "f.when(f.size(f.col('sorted_bucketized_tweet_word'))>0, f.col('sorted_bucketized_tweet_word').getItem(0)).otherwise(np.nan)",
             'second_used_word_bucket_id': "f.when(f.size(f.col('sorted_bucketized_tweet_word'))>1, f.col('sorted_bucketized_tweet_word').getItem(1)).otherwise(np.nan)"}, op='inline')
    proc.append_ops([op_feature_add_sorted_bucketized_tweet_word, op_feature_add_convert])

    # 5. transform
    t1 = timer()
    if sampleRatio != 1:
        df = df.sample(sampleRatio)
    df = proc.transform(df, name=output_name)
    t2 = timer()
    print("categorify and getMostAndSecondUsedWordBucketId took %.3f" % (t2 - t1))
    return (df, dict_dfs)


In [37]:
def TargetEncodingFeatures(df, proc, gen_dict, mode):   
    if mode == 'stage1':
        features = TE_col_features_stage1
        prefix = ''
    elif mode == 'stage2':
        features = TE_col_features_stage2
        prefix = 'stage2_'
    else:
        raise NotImplementedError("mode need to be train or valid")

    targets = ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp']
    y_mean_all = []
    
    t1 = timer()
    if gen_dict:
        for tgt in targets:
            tmp = df.groupBy().mean(tgt).collect()[0]
            y_mean = tmp[f"avg({tgt})"]
            y_mean_all.append(y_mean)
        schema = t.StructType([t.StructField(tgt, t.FloatType(), True) for tgt in targets])
        y_mean_all_df = proc.spark.createDataFrame([tuple(y_mean_all)], schema)
        y_mean_all_df.write.format("parquet").mode("overwrite").save(
            "%s/%s/%s/targets_mean" % (proc.path_prefix, proc.current_path, proc.dicts_path))
    y_mean_all_df = proc.spark.read.parquet(
        "%s/%s/%s/targets_mean" % (proc.path_prefix, proc.current_path, proc.dicts_path))

    te_train_dfs = []
    te_test_dfs = []
    for c in features:
        target_tmp = targets
        out_name = ""
        if str(c) in TE_col_excludes:
            target_tmp = []
            for tgt in targets:
                if tgt not in TE_col_excludes[c]:
                    target_tmp.append(tgt)
        out_col_list = []
        for tgt in target_tmp:
            if isinstance(c, list):
                out_col_list.append(prefix + 'GTE_'+'_'.join(c)+'_'+tgt)
                out_name = prefix + 'GTE_'+'_'.join(c)
            else:
                out_col_list.append(prefix + f'TE_{c}_{tgt}')
                out_name = prefix + f'TE_{c}'
        if mode == "stage1":
            threshold = TE_col_features_stage1_threshold[out_name]
        else:
            threshold = 0
        if gen_dict:
            start = timer()
            encoder = TargetEncoder(proc, c, target_tmp, out_col_list, out_name, out_dtype=t.FloatType(), y_mean_list=y_mean_all,threshold=threshold)
            te_train_df, te_test_df = encoder.transform(df)
            te_train_dfs.append({'col_name': ['fold'] + (c if isinstance(c, list) else [c]), 'dict': te_train_df})
            te_test_dfs.append({'col_name': c, 'dict': te_test_df})
            print(f"generating target encoding for %s upon %s took %.1f seconds"%(str(c), str(target_tmp), timer()-start))
        else:
            te_train_path = "%s/%s/%s/train/%s" % (proc.path_prefix, proc.current_path, proc.dicts_path, out_name)
            te_test_path = "%s/%s/%s/test/%s" % (proc.path_prefix, proc.current_path, proc.dicts_path, out_name)               
            te_train_dfs.append({'col_name': ['fold'] + (c if isinstance(c, list) else [c]), 'dict': proc.spark.read.parquet(te_train_path)})
            te_test_dfs.append({'col_name': c, 'dict': proc.spark.read.parquet(te_test_path)})
    t2 = timer()
    print("Generate encoding feature totally took %.3f" % (t2 - t1))

    return (te_train_dfs, te_test_dfs, y_mean_all_df)


In [41]:
def mergeTargetEncodingFeatures(df, te_train_dfs, proc, output_name, mode):
    feature_list = final_feature_list_stage1 if mode == 'stage1' else final_feature_list_stage2

    # merge dicts to original table
    op_merge_to_train = ModelMerge(te_train_dfs)
    proc.reset_ops([op_merge_to_train])
    
    # select features
    op_select = SelectFeature(feature_list)
    proc.append_ops([op_select])

    t1 = timer()
    df = proc.transform(df, name=output_name)
    t2 = timer()
    print("Merge Target Encoding Features took %.3f" % (t2 - t1))
    
    return df



In [31]:
############# decode data
df = df.withColumn("fold", f.round(f.rand(seed=42)*(5-1)).cast("int"))
df = decodeBertTokenizerAndExtractFeatures(df, proc, output_name="train1_decode")
print("data decoded!")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

save data to .//data//train1_decode
BertTokenizer decode and feature extacting took 37.148
data decoded!


In [36]:
############# categorify data
df, dict_dfs = categorifyFeatures(df, proc, output_name="train2_categorify", gen_dict=True)
print("data categorified!")


save data to .//data//recsys_dicts//tweet
save data to .//data//recsys_dicts//mention
Generate Dictionary took 5.268
save data to .//data//train2_categorify
categorify and getMostAndSecondUsedWordBucketId took 8.187
data categorified!


In [40]:
############# target encoding
te_train_dfs, te_test_dfs, y_mean_all_df = TargetEncodingFeatures(df, proc, gen_dict=True,mode="stage1")
print("data encoded!")

generating target encoding for engaged_with_user_id upon ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp'] took 3.8 seconds
generating target encoding for language upon ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp'] took 2.1 seconds
generating target encoding for dt_dow upon ['retweet_timestamp'] took 1.6 seconds
generating target encoding for tweet_type upon ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp'] took 2.1 seconds
generating target encoding for most_used_word_bucket_id upon ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp'] took 2.2 seconds
generating target encoding for second_used_word_bucket_id upon ['reply_timestamp', 'retweet_timestamp', 'retweet_with_comment_timestamp', 'like_timestamp'] took 2.0 seconds
generating target encoding for mentioned_count upon ['reply_timestamp', 'retweet_timestamp', 'r

In [42]:
############# merge target encoding only for selected data
df = mergeTargetEncodingFeatures(df, te_train_dfs, proc, output_name="stage1_train",mode="stage1")
print("data merged!")

save data to .//data//stage1_train
Merge Target Encoding Features took 11.819
data merged!
